In [1]:
import kagglehub
import pytesseract
from PIL import Image
import requests
import pandas as pd
from transformers import AutoProcessor, BlipForConditionalGeneration
import ollama 

pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe' # my dir where i installed tesseract

# Download latest version
path = kagglehub.dataset_download("akuppps/dankmemes-reddit-top-comments")

print("Path to dataset files:", path)

C:\Users\Leander Antony\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: C:\Users\Leander Antony\.cache\kagglehub\datasets\akuppps\dankmemes-reddit-top-comments\versions\1


In [2]:
df = pd.read_csv("C:\\Users\\Leander Antony\\.cache\\kagglehub\\datasets\\akuppps\\dankmemes-reddit-top-comments\\versions\\1\\dankmemes_Top_Comments.csv")

In [3]:
df.sample(10)

,Unnamed: 0,title,score,id,url,comms_num,created,body,timestamp
1418,1418,Comment,1,i39bsic,NaN,0,1.649014e+09,Oh bruh,2022-04-03 15:21:11
329,329,Let's go find some feathers,13531,th1glp,https://i.redd.it/094tpclea5o81.gif,60,1.647611e+09,NaN,2022-03-18 09:35:02
1075,1075,Comment,1,i39uuas,NaN,0,1.649022e+09,—(bots)—,2022-04-03 17:34:27
1880,1880,Comment,1,i38szvn,NaN,0,1.649006e+09,**Read this entire comment before modmailing u...,2022-04-03 13:12:43
62,62,RIP YouTube Vanced,48555,tdqfph,https://i.imgur.com/nEOCAGd.png,2372,1.647237e+09,NaN,2022-03-14 01:54:19
1891,1891,Comment,1,i38se1f,NaN,0,1.649006e+09,This submission has been removed. As an anti-...,2022-04-03 13:08:31
501,501,It do be like that,8343,trblfh,https://i.redd.it/ij8x3uzu4dq81.gif,35,1.648577e+09,NaN,2022-03-29 14:06:36
144,144,yes .,27285,tatwih,https://i.redd.it/6itgcg5bkim81.gif,181,1.646900e+09,NaN,2022-03-10 03:05:29
1288,1288,Comment,1,i39jecw,NaN,0,1.649017e+09,Drink,2022-04-03 16:13:22
449,449,HAHAHAHAHA BECAUSE THEY'RE UPSIDE DOWN,9784,toc0zx,https://i.redd.it/6pi0ax1nvmp81.gif,38,1.648259e+09,NaN,2022-03-25 21:48:31


In [4]:
len(df["title"])

1898

In [5]:
df.isna().sum()

Unnamed: 0      0
title           0
score           0
id              0
url           899
comms_num       0
created         0
body          999
timestamp       0
dtype: int64

In [6]:
df.dropna(subset=["url"], inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 999 entries, 0 to 998
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  999 non-null    int64  
 1   title       999 non-null    object 
 2   score       999 non-null    int64  
 3   id          999 non-null    object 
 4   url         999 non-null    object 
 5   comms_num   999 non-null    int64  
 6   created     999 non-null    float64
 7   body        0 non-null      object 
 8   timestamp   999 non-null    object 
dtypes: float64(1), int64(3), object(5)
memory usage: 78.0+ KB


In [7]:
urls = df.url

In [8]:
urls[1:9]

1    https://i.redd.it/cmem70sowcm81.gif
2    https://i.redd.it/5kxyntrizhm81.gif
3    https://i.redd.it/p4rc3d0yl1q81.jpg
4    https://i.redd.it/ycsi7qy775q81.gif
5    https://i.redd.it/70uum4rpxll81.gif
6    https://i.redd.it/fepe187jfjn81.gif
7    https://i.redd.it/to10s17e2bp81.gif
8    https://i.redd.it/z3i1fbc6kfq81.gif
Name: url, dtype: object

In [9]:
def extract_text(url):
    image = Image.open(requests.get(url, stream=True).raw)
    text = pytesseract.image_to_string(image)
    return text


def image_to_text(url):
    processor = AutoProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
    model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")
    image = Image.open(requests.get(url, stream=True).raw)
    text = "A picture of"
    
    inputs = processor(images=image, text=text, return_tensors="pt")
    
    outputs = model.generate(**inputs)
    caption = processor.tokenizer.decode(outputs[0], skip_special_tokens=True)
    return caption


def generate_caption(extracted_text, text_from_image):
    prompt = f"""you are a humoros meme caption generator. generate one caption for this text extracted from the meme {extracted_text} 
    and this description of the image {text_from_image}' """
    response = ollama.chat(model="mistral", messages=[{"role": "user", "content": prompt}])
    return response['message']['content']


In [10]:
def pipeline(url):
    extracted_text = extract_text(url)
    text_from_image = image_to_text(url)
    generated_caption = generate_caption(extracted_text, text_from_image)
    return generated_caption

In [11]:
for i in range(7):
    print(urls[i])
    captions = pipeline(urls[i])
    print(captions)

https://i.redd.it/zysreb6cv3q81.gif


C:\Users\Leander Antony\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\generation\utils.py:1259: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


 "When you think you're the funniest in the room, but then you see your wife's reaction..."
   "And then there's your wife, silently judging your humor level on a scale of 1 to 'Oh my God, did I really marry this guy?'"
https://i.redd.it/cmem70sowcm81.gif
 "When you thought your math skills would save the launch, but you were wrong, Captain Rocketry!"
https://i.redd.it/5kxyntrizhm81.gif
 USA: "When you realize they also want your meme game..." (Picture of the man in a suit and tie shifty-eyed, hiding behind his laptop screen)
https://i.redd.it/p4rc3d0yl1q81.jpg
 "Will Smith, when someone tells a harmless joke about his choice of toilet paper brands... Still laughing, because it's the principle!"
https://i.redd.it/ycsi7qy775q81.gif
 "Looks like they're about to kick off some laughs instead of goals at the Oscars!"
https://i.redd.it/70uum4rpxll81.gif
 "When you realize Russia just set a new world record for 'Most War Crimes Per Week' using nothing but the Reface App and a whole lotta 'ch